In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from bert_score import score
import re
from huggingface_hub import login


c:\Users\Daniel\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
pd.read_csv("../../tech_terms_sub.csv")

,Term,Context Sentence,Prompt,Ground Truth
0,Containerization,Modern applications often rely on containeriza...,"Define the term ""Containerization"" in one sent...",Containerization is a software deployment meth...
1,Inheritance,This software module uses inheritance to share...,"Define the term ""Inheritance"" in one sentence ...",Inheritance is an object-oriented programming ...
2,Latency,Reducing latency is critical in real-time comm...,"Define the term ""Latency"" in one sentence base...",Latency refers to the delay between a user's a...
3,Orchestration,The team implemented orchestration tools to ma...,"Define the term ""Orchestration"" in one sentenc...",Orchestration refers to the automated coordina...
4,Pipeline,Data scientists configured a pipeline to autom...,"Define the term ""Pipeline"" in one sentence bas...",A pipeline is a sequence of data processing st...
5,Load Balancing,The web service uses load balancing to handle ...,"Define the term ""Load Balancing"" in one senten...",Load balancing is the process of distributing ...
6,Endpoint,Each API must define at least one secure endpo...,"Define the term ""Endpoint"" in one sentence bas...",An endpoint is a specific URL or interface in ...
7,Cache,The app stores session data in the cache for f...,"Define the term ""Cache"" in one sentence based ...",A cache is a temporary storage layer that stor...
8,Stack,Their backend stack includes a database and a ...,"Define the term ""Stack"" in one sentence based ...",A stack in technology refers to a collection o...
9,Fork,The developer decided to fork the repository t...,"Define the term ""Fork"" in one sentence based o...",To fork a repository means to create a persona...


In [ ]:
login("--------------")  # Replace with your Hugging Face token

In [7]:
df = pd.read_csv("../../tech_terms_sub.csv") 
required_cols = ["Term", "Context Sentence", "Prompt", "Ground Truth"]
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise ValueError(f"Missing columns: {missing_cols}")

df = df.dropna(subset=["Prompt", "Ground Truth"]).reset_index(drop=True)

In [5]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    do_sample=False
)

c:\Users\Daniel\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Daniel\.cache\huggingface\hub\models--meta-llama--Llama-3.1-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.38s/it]
Some p

In [8]:
outputs = []
for prompt in df["Prompt"]:
    try:
        result = generator(prompt)[0]["generated_text"]
        response = result.replace(prompt, "").strip()
    except Exception as e:
        response = f"[ERROR: {str(e)}]"
    outputs.append(response)

df["Llama_8B_Output"] = outputs

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

In [9]:
df

,Term,Context Sentence,Prompt,Ground Truth,Llama_8B_Output
0,Containerization,Modern applications often rely on containeriza...,"Define the term ""Containerization"" in one sent...",Containerization is a software deployment meth...,Containerization is a lightweight and portable...
1,Inheritance,This software module uses inheritance to share...,"Define the term ""Inheritance"" in one sentence ...",Inheritance is an object-oriented programming ...,"The context sentence is: ""The inheritance of t..."
2,Latency,Reducing latency is critical in real-time comm...,"Define the term ""Latency"" in one sentence base...",Latency refers to the delay between a user's a...,Latency is the time delay between the moment a...
3,Orchestration,The team implemented orchestration tools to ma...,"Define the term ""Orchestration"" in one sentenc...",Orchestration refers to the automated coordina...,"The context sentence is: ""The composer's use o..."
4,Pipeline,Data scientists configured a pipeline to autom...,"Define the term ""Pipeline"" in one sentence bas...",A pipeline is a sequence of data processing st...,"The context sentence is: ""The pipeline will be..."
5,Load Balancing,The web service uses load balancing to handle ...,"Define the term ""Load Balancing"" in one senten...",Load balancing is the process of distributing ...,"The context sentence is: ""The load balancer is..."
6,Endpoint,Each API must define at least one secure endpo...,"Define the term ""Endpoint"" in one sentence bas...",An endpoint is a specific URL or interface in ...,"The context sentence is: ""The endpoint of the ..."
7,Cache,The app stores session data in the cache for f...,"Define the term ""Cache"" in one sentence based ...",A cache is a temporary storage layer that stor...,"The context sentence is: ""The browser's cache ..."
8,Stack,Their backend stack includes a database and a ...,"Define the term ""Stack"" in one sentence based ...",A stack in technology refers to a collection o...,"The context sentence is: ""The stack of books o..."
9,Fork,The developer decided to fork the repository t...,"Define the term ""Fork"" in one sentence based o...",To fork a repository means to create a persona...,"The context sentence is: ""The fork is a utensi..."


In [10]:
def clean_output(text):
    # Remove common prefixes and formatting
    text = re.sub(r"(?i)^answer:\s*", "", text)  # case-insensitive 'Answer:'
    text = re.sub(r"(?i)step\s*\d+/\d+\s*", "", text)  # Step 1/2 or 2/2 etc.
    text = re.sub(r"\n+", " ", text)  # Replace newlines with space
    return text.strip()

df["Llama_8B_Output"] = df["Llama_8B_Output"].apply(clean_output)

In [11]:
def deduplicate_sentences(text):
    sentences = list(dict.fromkeys(text.split('. ')))  # remove exact duplicates
    return '. '.join(sentences)

df["Llama_8B_Output"] = df["Llama_8B_Output"].apply(deduplicate_sentences)

In [ ]:
P, R, F1 = score(
    df["Llama_8B_Output"].tolist(),
    df["Ground Truth"].tolist(),
    lang="en",
    verbose=True
)

df["BERTScore_Precision"] = P
df["BERTScore_Recall"] = R
df["BERTScore_F1"] = F1

print(f"✅ Average BERTScore F1: {F1.mean():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\Daniel\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 1/1 [00:02<00:00,  2.60s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 85.21it/s]

done in 2.61 seconds, 3.83 sentences/sec
✅ Average BERTScore F1: 0.8499


In [ ]:
df.to_csv("llama8b_bert_score_results.csv", index=False)
print("Saved: llama8b_bert_score_results.csv")